In [ ]:
#@title Install Requirements
!pip install gradio pillow numpy

In [ ]:
#@title Start  WebUI

import gradio as gr
import os
import zipfile
from PIL import Image
import tempfile
import shutil
from typing import List, Tuple, Optional
import re

class GradioImageConverter:
    def __init__(self):
        self.temp_dir = tempfile.mkdtemp()

    def convert_images(self,
                      files: List[str],
                      output_format: str,
                      rename_mode: str,
                      custom_name: str = "",
                      start_number: int = 1) -> Tuple[str, str]:
        """
        Convert images and apply renaming

        Args:
            files: List of file paths
            output_format: Target format (png, jpg, etc.)
            rename_mode: 'keep_original', 'custom', 'numbered', 'custom_numbered'
            custom_name: Custom base name for renaming
            start_number: Starting number for numbered files
        """
        if not files:
            return None, "❌ No files provided"

        try:
            # Create output directory
            output_dir = os.path.join(self.temp_dir, "converted_images")
            os.makedirs(output_dir, exist_ok=True)

            converted_files = []
            errors = []

            for i, file_path in enumerate(files):
                try:
                    # Generate output filename based on rename mode
                    original_name = os.path.splitext(os.path.basename(file_path))[0]

                    if rename_mode == "keep_original":
                        output_name = original_name
                    elif rename_mode == "custom":
                        if not custom_name.strip():
                            output_name = f"image_{i+1}"
                        else:
                            output_name = custom_name.strip()
                            if len(files) > 1:
                                output_name = f"{output_name}_{i+1}"
                    elif rename_mode == "numbered":
                        output_name = f"{start_number + i:03d}"
                    elif rename_mode == "custom_numbered":
                        if not custom_name.strip():
                            base_name = "image"
                        else:
                            base_name = custom_name.strip()
                        output_name = f"{base_name}_{start_number + i:03d}"
                    else:
                        output_name = original_name

                    # Clean filename (remove invalid characters)
                    output_name = re.sub(r'[<>:"/\\|?*]', '_', output_name)
                    output_file = os.path.join(output_dir, f"{output_name}.{output_format.lower()}")

                    # Convert image
                    with Image.open(file_path) as img:
                        # Handle different formats
                        if output_format.lower() in ['jpg', 'jpeg']:
                            # Convert RGBA to RGB for JPEG
                            if img.mode in ('RGBA', 'LA', 'P'):
                                background = Image.new('RGB', img.size, (255, 255, 255))
                                if img.mode == 'P':
                                    img = img.convert('RGBA')
                                background.paste(img, mask=img.split()[-1] if img.mode == 'RGBA' else None)
                                img = background
                            img.save(output_file, 'JPEG', quality=95)
                        elif output_format.lower() == 'webp':
                            img.save(output_file, 'WEBP', quality=95)
                        else:
                            img.save(output_file, output_format.upper())

                    converted_files.append(output_file)

                except Exception as e:
                    errors.append(f"❌ {os.path.basename(file_path)}: {str(e)}")

            if not converted_files:
                return None, f"❌ No files converted successfully.\n\n" + "\n".join(errors)

            # Create ZIP file if multiple files
            if len(converted_files) > 1:
                zip_path = os.path.join(self.temp_dir, "converted_images.zip")
                with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                    for file_path in converted_files:
                        zipf.write(file_path, os.path.basename(file_path))

                success_msg = f"✅ Successfully converted {len(converted_files)} images to {output_format.upper()}"
                if errors:
                    success_msg += f"\n\n⚠️ Errors ({len(errors)}):\n" + "\n".join(errors[:5])
                    if len(errors) > 5:
                        success_msg += f"\n... and {len(errors)-5} more errors"

                return zip_path, success_msg
            else:
                # Single file
                success_msg = f"✅ Successfully converted 1 image to {output_format.upper()}"
                if errors:
                    success_msg += f"\n\n⚠️ Some files had errors:\n" + "\n".join(errors)

                return converted_files[0], success_msg

        except Exception as e:
            return None, f"❌ Error during conversion: {str(e)}"

    def update_rename_options(self, rename_mode: str):
        """Update visibility of rename options based on selected mode"""
        custom_visible = rename_mode in ["custom", "custom_numbered"]
        number_visible = rename_mode in ["numbered", "custom_numbered"]

        return (
            gr.update(visible=custom_visible),  # custom_name
            gr.update(visible=number_visible)   # start_number
        )

def create_interface():
    converter = GradioImageConverter()

    # Responsive CSS that adapts to screen size
    responsive_css = """
    /* Remove fixed width constraints */
    .gradio-container {
        max-width: none !important;
        width: 100% !important;
        padding: 1rem !important;
    }

    /* Responsive layout */
    @media (max-width: 768px) {
        .gradio-container {
            padding: 0.5rem !important;
        }

        /* Stack columns on mobile */
        .gr-row {
            flex-direction: column !important;
        }

        /* Full width components on mobile */
        .gr-column {
            width: 100% !important;
            margin-bottom: 1rem !important;
        }

        /* Adjust file input height on mobile */
        .file-preview {
            max-height: 250px !important;
        }

        /* Smaller text on mobile */
        .gr-markdown h1 {
            font-size: 1.5rem !important;
        }

        .gr-markdown h3 {
            font-size: 1.1rem !important;
        }

        /* Better button sizing */
        .gr-button {
            width: 100% !important;
            margin: 0.5rem 0 !important;
        }
    }

    /* Tablet styles */
    @media (min-width: 769px) and (max-width: 1024px) {
        .gradio-container {
            padding: 1rem !important;
        }

        .file-preview {
            max-height: 350px;
        }
    }

    /* Desktop styles */
    @media (min-width: 1025px) {
        .gradio-container {
            max-width: 1200px !important;
            margin: 0 auto !important;
        }

        .file-preview {
            max-height: 400px;
        }
    }

    /* General improvements */
    .file-preview {
        overflow-y: auto;
        border-radius: 0.5rem;
    }

    /* Better spacing */
    .gr-block {
        margin-bottom: 1rem;
    }

    /* Responsive table */
    .gr-markdown table {
        width: 100%;
        overflow-x: auto;
        display: block;
        white-space: nowrap;
    }

    @media (max-width: 768px) {
        .gr-markdown table {
            font-size: 0.8rem;
        }
    }
    """

    with gr.Blocks(
        title="Image Converter Pro",
        theme=gr.themes.Soft(),
        css=responsive_css
    ) as interface:

        gr.Markdown("""
        # 🖼️ Image Converter Pro

        **Convert images between formats with flexible renaming options**

        ✨ **Features:**
        - Convert between PNG, JPG, BMP, GIF, TIFF, WebP
        - Bulk conversion support
        - Flexible file renaming options
        - Download as ZIP for multiple files
        """)

        # Use responsive layout - no fixed scales
        with gr.Row():
            with gr.Column():
                # File input
                files_input = gr.File(
                    label="📁 Select Images",
                    file_count="multiple",
                    file_types=["image"],
                    height=200
                )

                # Format selection
                format_input = gr.Dropdown(
                    choices=["png", "jpg", "jpeg", "bmp", "gif", "tiff", "webp"],
                    value="png",
                    label="🎯 Output Format",
                    info="Choose the target image format"
                )

            with gr.Column():
                # Renaming options
                gr.Markdown("### 🏷️ File Renaming Options")

                rename_mode = gr.Radio(
                    choices=[
                        ("Keep original names", "keep_original"),
                        ("Custom name", "custom"),
                        ("Numbered (001, 002, ...)", "numbered"),
                        ("Custom name + numbers", "custom_numbered")
                    ],
                    value="keep_original",
                    label="Rename Mode",
                    info="Choose how to rename the converted files"
                )

                custom_name = gr.Textbox(
                    label="Custom Base Name",
                    placeholder="Enter custom name (e.g., 'vacation', 'project')",
                    visible=False,
                    info="Base name for your files"
                )

                start_number = gr.Number(
                    label="Starting Number",
                    value=1,
                    minimum=1,
                    maximum=9999,
                    visible=False,
                    info="Number to start counting from"
                )

        # Convert button - full width
        convert_btn = gr.Button(
            "🚀 Convert Images",
            variant="primary",
            size="lg"
        )

        # Output section - responsive
        with gr.Row():
            with gr.Column():
                output_file = gr.File(
                    label="📥 Download Converted Images",
                    visible=True
                )

            with gr.Column():
                result_text = gr.Textbox(
                    label="📊 Conversion Results",
                    lines=6,
                    max_lines=10,
                    interactive=False
                )

        # Examples section with responsive table
        gr.Markdown("""
        ### 💡 Renaming Examples:

        | Mode | Input Example | Output Example |
        |------|---------------|----------------|
        | **Keep original** | `photo1.jpg, photo2.jpg` | `photo1.png, photo2.png` |
        | **Custom** | Base: "vacation" | `vacation_1.png, vacation_2.png` |
        | **Numbered** | Start: 5 | `005.png, 006.png` |
        | **Custom + Numbers** | Base: "img", Start: 10 | `img_010.png, img_011.png` |

        ### 📱 Mobile Friendly
        This interface automatically adapts to your screen size for the best experience on any device!
        """)

        # Event handlers
        rename_mode.change(
            fn=converter.update_rename_options,
            inputs=[rename_mode],
            outputs=[custom_name, start_number]
        )

        convert_btn.click(
            fn=converter.convert_images,
            inputs=[
                files_input,
                format_input,
                rename_mode,
                custom_name,
                start_number
            ],
            outputs=[output_file, result_text]
        )

    return interface

def main():
    # Create and launch the interface
    interface = create_interface()

    # Launch with sharing enabled
    interface.launch(
        share=True,
        inline=False,
        debug=True
    )

if __name__ == "__main__":
    main()